<a href="https://colab.research.google.com/github/CRAUGUTH/nnProject/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Training Neural Network**


## Installing and Importing Libraries

In [ ]:
!pip install --upgrade spotipy torch pandas scikit-learn transformers librosa joblib

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
from google.colab import drive

## Defining the Model

Here is where the two different models are defined, the first being the default model with all the classifible attributes and the second being the model with the least amount of classifible attributes.

In [156]:
class MusicDataDefault(Dataset):
    def __init__(self, path):
        data = pd.read_csv(path)
        data = data.drop(['track_id'], axis=1)
        self.label_encoder = LabelEncoder()
        data['track_genre'] = self.label_encoder.fit_transform(data['track_genre'])
        self.features = data.drop(['track_genre'], axis=1).values.astype(np.float32)
        self.labels = data['track_genre'].values.astype(np.float32)
        self.scaler = StandardScaler()
        self.features = self.scaler.fit_transform(self.features)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.labels[idx]
        return x, y

The way that this model was determined was by taking the default model and removing one attribute at a time to see which attributes where the most important. During these test we had only trained 10 epochs in order to efficiently determine the accuracy of the models. Once the order of most important attribues was determined we added attributes back unitl the accuracy was the same, if not a bit higher, than the origainal model. This allowed us to obtain a model with the highest accuracy with the fewest number of classifible attributes.

In [157]:
class MDN(Dataset):
    def __init__(self, path):
        data = pd.read_csv(path)
        data = data.drop(['track_id', 'time_signature', 'liveness', 'key', 'mode',
                          'speechiness', 'energy', 'duration_ms', 'instrumentalness'], axis=1)
        self.label_encoder = LabelEncoder()
        data['track_genre'] = self.label_encoder.fit_transform(data['track_genre'])
        self.features = data.drop(['track_genre'], axis=1).values.astype(np.float32)
        self.labels = data['track_genre'].values.astype(np.float32)
        self.scaler = StandardScaler()
        self.features = self.scaler.fit_transform(self.features)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.labels[idx]
        return x, y

## Define Underlying MLP

In [158]:
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )


    def forward(self, x):
        return self.network(x)

### Mount Google Drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Junior/Semester_2/NN/Project/dataset.csv'

### Define Evaluate Function

In [160]:
# Function to evaluate model performance
def evaluate(model, data_loader):
    model.eval()
    total_correct = 0
    total_samples = 0
    all_predicted = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels.long()).sum().item()
            all_predicted.extend(predicted.view(-1).tolist())
            all_labels.extend(labels.long().view(-1).tolist())

    accuracy = (total_correct / total_samples) * 100
    f1 = f1_score(all_labels, all_predicted, average='weighted')
    return accuracy, f1

## Neural Network Training Sequence

In [179]:
# Load datasets
datasets = [MDN(data_path)]


for dataset in datasets:

    # Define best model and model accuracy
    best_val_accuracy = 0
    best_model_state = None

    # Splitting the dataset
    total_size = len(dataset)
    train_set_size = int(total_size * 0.6)
    val_set_size = int(total_size * 0.2)
    test_set_size = total_size - (train_set_size + val_set_size)
    train_set, val_set, test_set = random_split(dataset, [train_set_size, val_set_size, test_set_size])

    train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=True)
    val_loader = DataLoader(dataset=val_set, batch_size=64, shuffle=False)
    test_loader = DataLoader(dataset=test_set, batch_size=64, shuffle=False)

    # Model, loss, and optimizer
    input_dim = dataset.features.shape[1]
    num_classes = len(np.unique(dataset.labels))

    model = MLP(input_dim, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.3, patience=5)

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_correct += (predicted == labels.long()).sum().item()
            total_samples += labels.size(0)

        train_accuracy = (total_correct / total_samples) * 100
        val_accuracy, val_f1 = evaluate(model, val_loader)

        # Adjust the learning rate based on validation accuracy
        scheduler.step(val_accuracy)

        # Keeps the most accurate model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()

        # Print training and validation results
        # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/total_samples:.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%, Validation F1: {val_f1:.4f}')

    # After training is complete, restore the best model state
    model.load_state_dict(best_model_state)

    # Evaluate on the test set without loading any saved state
    test_accuracy, test_f1 = evaluate(model, test_loader)
    print(f'Dataset {dataset.__class__.__name__}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')

Dataset MDN, Training Accuracy: 71.50%, Validation Accuracy: 71.75%, Test Accuracy: 72.67%


### Save Model

In [ ]:
# Save the model and preprocessors
torch.save(model.state_dict(), 'song_genre_classifier.pth')
joblib.dump(dataset.label_encoder, 'label_encoder.joblib')
joblib.dump(dataset.scaler, 'scaler.joblib')

# **Music Classification**

## Obtain Personal Spotify Playlist

Get new access token for Spotify account. In order to use this new token you must run the cell, click the link, and then copy the text after 'code='

In [181]:
client = spotipy.oauth2.SpotifyOAuth(
    client_id='cce8687d5317494cb81ff7731ccd9a2b',
    client_secret='9646e0745dde4bddb7c67b29431a61f0',
    redirect_uri='https://example.com/callback/',
    scope='playlist-modify-private'
)

sp = spotipy.Spotify(auth_manager=client)
authorization_url = client.get_authorize_url()
print(authorization_url)

https://accounts.spotify.com/authorize?client_id=cce8687d5317494cb81ff7731ccd9a2b&response_type=code&redirect_uri=https%3A%2F%2Fexample.com%2Fcallback%2F&scope=playlist-modify-private


This is where you authorize your account, the code is where you paste the text from the link above

In [ ]:
code = "AQAgXDjXngKs5FyHLs-hoaao4aIQMOJVkA6iqqjQAohdkmNhzuinHaoly0MGby-VVvDAvf8hXtbnwuj8JltaVMQkbk-bppscOq02c_fXRUL3mGpD7ogPNcf7fFp8JqdadpFguhOXtT_rnX__uyG6OcC3l4QgioVXhp6maPs7VWwloRUjHIS2CJqTZR-SCLgl3g-XkTpkHTYiMQ"
client.get_access_token(code)

Getting playlist from my account we want to classify which is the USA Top Songs

In [183]:
# Function to get all tracks from a playlist
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = [item['track']['id'] for item in results['items'] if item['track']]
    while results['next']:
        results = sp.next(results)
        tracks.extend([item['track']['id'] for item in results['items'] if item['track']])
    return tracks

# Get all tracks from a specific playlist
playlist_id = '3qILVAqgqMNH18dPJPvan1'
song_ids = get_playlist_tracks(playlist_id)

Here we are creating a dictionary of the songs within that playlist. The dictionary contains the key, which is the song_id, and then the song name, artist, and genre.

In [ ]:
# Load the model and preprocessors
model = MLP(input_dim, num_classes)
model.load_state_dict(torch.load('song_genre_classifier.pth'))
model.eval()

scaler = joblib.load('scaler.joblib')
label_encoder = joblib.load('label_encoder.joblib')

results = {}

for song_id in song_ids:
    song_features = sp.audio_features(song_id)[0]
    track_info = sp.track(song_id)
    song_name = track_info['name']
    artist_name = track_info['artists'][0]['name']
    popularity = track_info['popularity']

    # Ensure these features match the training set's features
    features_list = ['danceability', 'loudness', 'acousticness', 'valence', 'tempo']
    features_values = [song_features[feature] for feature in features_list]
    features_values.append(popularity)
    features_array = np.array([features_values], dtype=np.float32)
    scaled_features = scaler.transform(features_array)

    input_tensor = torch.tensor(scaled_features, dtype=torch.float32)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        genre = label_encoder.inverse_transform(predicted.numpy())[0]

    results[song_id] = {
        'song_name': song_name,
        'artist_name': artist_name,
        'genre': genre
    }

for song_id, info in results.items():
    print(f"{info['song_name']} by {info['artist_name']}, Genre: {info['genre']}")
